For my hypothesis test, I am hypothesizing that various columns will have adverse effects on whether or not someone will have diabetes. I am going to run *** seperate tests in order to test the if the values in the *** columns can indicate whether someone is more or less likely to get diabetes.

**H0**: The mean of the values in *** columns will be roughly the same for patients with diabetes and those without.

**H1**: The mean of the values in the *** columns will be significantly different for the patients with diabetes compared to patients without diabetes.


**Results**

According the the p-values, BMI, physical health and mental health are all strong indicators of a patients alikelihood to end up with diabetes. All of the columns showed a p-value of 0 which indicates that there is very significant differences in their values for patients with diabetes compared to those without. 


Upon further analysis, it may appear that columns such as the mental health columns might not directly effect whether or not a patient has diabetes. Instead, it is likely that mental health is conditionally indepedent from diabetes, given other factors such as physical health or BMI.

In [7]:
import pandas as pd
import scipy.stats as stats

df = pd.read_csv('diabetes.csv')

categorical_vars = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack',
                    'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare',
                    'NoDocbcCost', 'DiffWalk', 'Sex']

chi_results = {}

for var in categorical_vars:
    contingency_table = pd.crosstab(df[var], df['Diabetes_binary'])
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    chi_results[var] = p

continuous_vars = ['BMI', 'MentHlth', 'PhysHlth']

t_results = {}

for var in continuous_vars:
    group1 = df[df['Diabetes_binary'] == 0][var]
    group2 = df[df['Diabetes_binary'] == 1][var]
    t_stat, p_val = stats.ttest_ind(group1, group2, equal_var=True, permutations=1000, alternative='less')
    t_results[var] = p_val

print("Chi-Square Test Results:")
for var, p_val in chi_results.items():
    print(f"{var}: p-value = {p_val}")

print("\nT-Test Results:")
for var, p_val in t_results.items():
    print(f"{var}: p-value = {p_val}")


Chi-Square Test Results:
HighBP: p-value = 0.0
HighChol: p-value = 0.0
CholCheck: p-value = 3.7513988789188705e-233
Smoker: p-value = 8.640171809942516e-206
Stroke: p-value = 0.0
HeartDiseaseorAttack: p-value = 0.0
PhysActivity: p-value = 0.0
Fruits: p-value = 1.0881213885155634e-93
Veggies: p-value = 1.4630291539062376e-178
HvyAlcoholConsump: p-value = 1.8659318101056406e-181
AnyHealthcare: p-value = 2.986181073970452e-16
NoDocbcCost: p-value = 2.218395410122378e-56
DiffWalk: p-value = 0.0
Sex: p-value = 2.1098747365768505e-56

T-Test Results:
BMI: p-value = 0.0
MentHlth: p-value = 0.0
PhysHlth: p-value = 0.0


For the second hypothesis test, I am going to test the validity of the first hypothesis test to see if mental health really does cause diabetes or both are caused by physical health. I am going to find the correlation between mental and physical health by using a linear regression model between mental health and physical health values, then I will fit a linear regression model for diabetes and physical health. Finally I will find the correlation between the residuals to conclude whether or not there is conditional independence.

**H0**: Mental health and diabetes are conditionally independent given phyiscal health.

**H1**: Mental health and diabetes are not conditionally independent given phyiscal health.


**Results**

The correlation was not strong enough and therefore we must reject the null hypothesis. Mental health and diabetes are NOT conditionally independent given phyiscal health.

In [10]:
import statsmodels.api as sm
from scipy.stats import pearsonr

X1 = sm.add_constant(df['PhysHlth'])
model1 = sm.OLS(df['MentHlth'], X1).fit()
residuals_menthlth = model1.resid

X2 = sm.add_constant(df['PhysHlth'])
model2 = sm.OLS(df['Diabetes_binary'], X2).fit()
residuals_diabetes = model2.resid

correlation, p_value = pearsonr(residuals_menthlth, residuals_diabetes)

print(f"Correlation: {correlation}")
print(f"P-value: {p_value}")


Correlation: 0.009470046225059
P-value: 1.8438236078563135e-06
